# Proxy Rotation (Note: Use EliteProxies)

In [8]:
import requests
from bs4 import BeautifulSoup

In [3]:
res = requests.get('https://free-proxy-list.net')
content = BeautifulSoup(res.text, 'lxml')
table = content.find('table')
rows = table.find_all('tr')
cols = [[col.text for col in row.find_all('td')] for row in rows]

proxies = []
proxy_index = 0

for col in cols:
    try:
        if col[4] == 'elite proxy' and col[6] == 'yes':
            proxies.append('https://' + col[0] + ':' + col[1])
    except:
        pass
# print (proxies)

def fetch(url, params):
    global proxy_index
    
    while proxy_index < len(proxies):
        try:
            print('Trying proxy:', proxies[proxy_index])
            res = requests.get(url, proxies={'https': proxies[proxy_index]}, params=params, timeout=5)
            return res
            
        except:
            print('Bad proxy...')
            proxy_index += 1

for page in range(0, 4):
    params = {'page': page}
    res = fetch('https://www.google.com/', params=params)
    proxy_index += 1
    print('ip', res.json()['ip'])
    print('url:', res.json()['url'])  

Trying proxy: https://47.240.160.90:10001
Bad proxy...
Trying proxy: https://195.158.30.232:3128
Bad proxy...
Trying proxy: https://5.161.105.105:80
Bad proxy...
Trying proxy: https://43.225.23.132:80
Bad proxy...
Trying proxy: https://150.136.108.121:3128
Bad proxy...
Trying proxy: https://85.25.91.141:15333
Bad proxy...
Trying proxy: https://85.195.104.71:80
Bad proxy...
Trying proxy: https://38.108.119.176:59394
Bad proxy...
Trying proxy: https://50.231.95.3:8080
Bad proxy...
Trying proxy: https://18.230.58.67:3128
Bad proxy...
Trying proxy: https://78.46.123.202:80
Bad proxy...
Trying proxy: https://5.189.184.6:80
Bad proxy...
Trying proxy: https://1.10.141.220:54620
Bad proxy...
Trying proxy: https://154.66.210.1:8080
Bad proxy...
Trying proxy: https://151.106.17.122:1080
Bad proxy...
Trying proxy: https://151.106.17.125:1080
Bad proxy...
Trying proxy: https://151.106.17.123:1080
Bad proxy...


AttributeError: 'NoneType' object has no attribute 'json'

# Word Combinations (of 20 Words Given)

In [24]:
import requests
import time
from bs4 import BeautifulSoup

In [25]:
words2Scrap=["Table", "Fog", "Wikipedia", "Empire", "Ruin", "Era", "English", "Library", "Twin", "Tower", "Book", "Art",
             "Science", "Poor", "Rich", "Dad", "Heart", "Power", "Medicine", "Borrow" ]

In [26]:
from itertools import combinations

#Source: https://stackoverflow.com/questions/464864/how-to-get-all-possible-combinations-of-a-list-s-elements#answer-32555776
def combs(x):
    return [c for i in range(len(x)+1) for c in combinations(x,i)]

In [27]:
lister= list(combs(words2Scrap))

In [28]:
len(lister)

1048576

In [29]:
def formatSearchTerm4url(typle,index):
    searchTerm=""
    for i,d in enumerate(list(typle[index])):
        if (i!=0):
            searchTerm+="+"+str(d)
        else:
            searchTerm=d
    urlMain= f"https://openlibrary.org/search?q=paper+trains&mode=everything"
    return urlMain


In [30]:
def getRobotDelay(num_runs,delayTime):
    #every 10 runs/pages or if first time, get delay from robots.txt
    if (num_runs%20==0 or delayTime==0):
        res = requests.get('https://openlibrary.org/robots.txt')
        content = BeautifulSoup(res.text, 'lxml')

        delayTime = float(str(content).split("\nCrawl-delay: ")[-1].split("\n")[0])
        print("just got robot.txt... current delay time: ",delayTime)
        return delayTime
    else:
        return delayTime
        

In [31]:
import requests
from bs4 import BeautifulSoup
import time
import numpy

def getLinksOfPageURL(lister, index,urls, num_runs, delayTime):
    delayTime = getRobotDelay(num_runs,delayTime)
    
    if (type(lister) is not str):
        URL =formatSearchTerm4url(lister,index)
        page = requests.get(URL)

        soup = BeautifulSoup(page.content, "html.parser")
        job_elements = soup.find(class_="pagination")

        page_no=0

        if (lister[index]!=()):
            pages= job_elements.find_all(class_="ChoosePage",href=True)

            print(f"\n\n\n\n\n\nURL of Index {index}...")
            for indexPage,data in enumerate(pages):
                if (indexPage==0):
                    #then add page 1 (since there's page 2)
                    url="https://openlibrary.org"+data['href'][:len(data['href'])-1]+"1"
                    page_no=1
                    print ("Found URL:", (url))
                    urls.append(url)
                    
                    #starts with page 2 thus add this too
                    url="https://openlibrary.org"+data['href']
                    print ("Found URL:",url )
                    page_no=int(data.text)
                    urls.append(url)
                    
                else:
                    try:
                        url="https://openlibrary.org"+data['href']
                        print ("Found URL:",url )
                        page_no=int(data.text)
                        urls.append(url)
                        
                        print(len(urls))
                    except:
                        pass
                    

            if (page_no!=0):
                url="https://openlibrary.org"+data['href'][:len(data['href'])-1]+str(page_no)
                print("zzzzz Sleeping for ",delayTime," sec(s), from Robot.txt")
                time.sleep(delayTime)
                print("ooooo Awake Now")
                
                getLinksOfPageURL(url,page_no,urls,int(page_no),delayTime)

        else:
            print(f"NOTE: Lister's Index of {index} has ZERO elements!")
            return []
    
    else:
        delayTime = getRobotDelay(num_runs,delayTime)
        page = requests.get(lister)
        
        print("Total Pages So Far: ", index)
        page_no=index

        try:
            page = requests.get(lister)
            soup = BeautifulSoup(page.content, "html.parser")
            job_elements = soup.find(class_="pagination")

            pages= job_elements.find_all(class_="ChoosePage",href=True)

            print(f"\nFrom Page {index}...")
            for indexPage,data in enumerate(pages):
                    if ("Next" in data.text or "First" in data.text or "Previous" in data.text):
                        pass
                    elif (int(data.text)>page_no):
                        url="https://openlibrary.org"+data['href']
                        print ("Found URL:",url )
                        urls.append(url)
                        print(len(urls))
                        page_no=int(data.text)

            print("zzzzz Sleeping for ",delayTime," sec(s), from Robot.txt")
            time.sleep(delayTime)
            print("ooooo Awake Now")
            
            getLinksOfPageURL(url,page_no,urls,int(page_no),delayTime)
        except:
            print("End Of the Line.... No More Next Page")
            print(urls)
            return urls
        return urls

In [32]:
formatSearchTerm4url(lister,0)

'https://openlibrary.org/search?q=paper+trains&mode=everything'

# Get Each Page's URL and store in CSV

In [ ]:
urlFile=[]
for i in range(0,len(lister)):
    #start with number of runs 0
    urlFile.append(getLinksOfPageURL(lister,i,urlFile,i,0))
    urlFile = [x for x in urlFile if x != []]
    print("TOTAL LENGTH OF URLS COLLECTED for keywords\'",lister[i],"\' :",len(urlFile))
    
    a = numpy.asarray(urlFile)
    numpy.savetxt(f"URLScrapedKeyWordsLinks.csv", a, delimiter=",", fmt='%s')
#     numpy.savetxt(f"URLScrapedKeyWords_WITH_{len(urlFile)}_links.csv", a, delimiter=",", fmt='%s')

        
#NOTE: total 2865 pages

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


just got robot.txt... current delay time:  0.5
NOTE: Lister's Index of 0 has ZERO elements!
TOTAL LENGTH OF URLS COLLECTED for keywords' () ' : 0
just got robot.txt... current delay time:  0.5






URL of Index 1...
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=1
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=2
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=3
3
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=4
4
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=5
5
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=6
6
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=7
7
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=8
8
Found URL: https://openlibrary.org/search?q=paper+trains&mode=everything&page=9
9
Found URL: https://openlibrary.org/search?q=paper

# Get Each Page's URL from CSV

In [41]:
import csv
pageURLs = []
with open('URLScrapedKeyWordsLinks.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are\n{", ".join(row)}')
            pageURLs.append(row)
            line_count += 1
        else:
            print(f'{row[0]}')
            line_count += 1
            pageURLs.append(row)
    print(f'Processed {line_count} lines.')

Column names are
https://openlibrary.org/search?q=paper+trains&mode=everything&page=1
https://openlibrary.org/search?q=paper+trains&mode=everything&page=2
https://openlibrary.org/search?q=paper+trains&mode=everything&page=3
https://openlibrary.org/search?q=paper+trains&mode=everything&page=4
https://openlibrary.org/search?q=paper+trains&mode=everything&page=5
https://openlibrary.org/search?q=paper+trains&mode=everything&page=6
https://openlibrary.org/search?q=paper+trains&mode=everything&page=7
https://openlibrary.org/search?q=paper+trains&mode=everything&page=8
https://openlibrary.org/search?q=paper+trains&mode=everything&page=9
https://openlibrary.org/search?q=paper+trains&mode=everything&page=10
https://openlibrary.org/search?q=paper+trains&page=11&mode=everything
https://openlibrary.org/search?q=paper+trains&page=12&mode=everything
https://openlibrary.org/search?q=paper+trains&page=13&mode=everything
https://openlibrary.org/search?q=paper+trains&page=14&mode=everything
https://open

# Get Each Book's URL

In [132]:
delayTime = 0
bookURLs =[]
for index, item in enumerate(pageURLs):
    delayTime = getRobotDelay(index,delayTime)
    print("\n\nzzzzz Sleeping for ",delayTime," sec(s), from Robot.txt")
    time.sleep(delayTime)
    print("ooooo Awake Now")
            
    page = requests.get(pageURLs[index][0]) #url
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find('ul', class_='list-books')
    for item in results:
        if (item.find("h3") == -1):
            pass
        else:
            url = "https://openlibrary.org"+item.find("a")['href']
            print(url)
            bookURLs.append(url)
    if (index ==2):
        break

just got robot.txt... current delay time:  0.5


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
https://openlibrary.org/works/OL6729940W
https://openlibrary.org/works/OL6729938W
https://openlibrary.org/works/OL6729935W
https://openlibrary.org/works/OL6729930W
https://openlibrary.org/works/OL6729928W
https://openlibrary.org/works/OL6729927W
https://openlibrary.org/works/OL6729909W
https://openlibrary.org/works/OL25294299W
https://openlibrary.org/works/OL24162515W?edition=ia%3Apapertrainechoes0000phei
https://openlibrary.org/works/OL6729934W
https://openlibrary.org/works/OL6729921W
https://openlibrary.org/works/OL6729933W
https://openlibrary.org/works/OL6729924W
https://openlibrary.org/works/OL18883167W
https://openlibrary.org/works/OL11929301W
https://openlibrary.org/works/OL18883166W
https://openlibrary.org/works/OL25547462W
https://openlibrary.org/works/OL6729929W
https://openlibrary.org/works/OL6729910W
https://openlibrary.org/works/OL6729916W


zzzzz Sleeping for  0

In [426]:
def getBookDetails(soup):
    dict1={}
    
    try:
        results = soup.find('div', class_='workDetails')
    except:
        pass
    
    #get Book Title
    try:
        dict1['title'] = results.find("h1",class_="work-title").text
    except:
        pass
    
    #get Author of Book URL
    try:
        dict1["authorURL"] = "https://openlibrary.org"+results.find("h2",class_="edition-byline").find("a")['href']
    except:
        pass
    
    #get Book Date
    try:
        dict1["date"] = results.find("div",  class_="smallest").text.split(" | ")[0]
    except:
        pass
    
    #Get Subject(s) Name, URL
    try:
        dictNest={}
        subjects = results.find("div",  class_="subjects-content").find("span",class_="clamp").find_all("a")
        for index, subject in enumerate(subjects):
            dictNest[index] = {"name":subject.text, "href": "https://openlibrary.org"+subject['href']}
        dict1["subject"]=dictNest
    except:
        pass
    
    #Get Published Date
    publish_date = soup.find("div",class_="edition-omniline-item").find("span").text
    dict1["publish_date"] = publish_date
    
    
    #Book Details Section
    sections = soup.find("div",  class_="edition-info").find_all("div",class_="section")
    for section in sections:
        try:
            published_in = section.findChild("p").text
            dict1['published_in'] = published_in.replace("\n          ","")
        except:
            pass

            dewy = section.findChild("dl")
        try:


            try:
                for index,item in enumerate(dewy.find_all("dt")):
                    if ("OCLC/WorldCat" in dewy.find_all("dt")[index].text):
                        dict1["oclc_worldcat"] = dewy.find_all("dd")[index].text.replace("\n","").replace(" ","")
                    if ("ISBN" in dewy.find_all("dt")[index].text):
                        dict1["isbn"] = dewy.find_all("dd")[index].text.replace("\n","").replace(" ","")
                    if ("Open Library" in dewy.find_all("dt")[index].text):
                        dict1["open_library_id"] = dewy.find_all("dd")[index].text.replace("\n","").replace(" ","")

                    if ("Dewey Decimal Class" in dewy.find_all("dt")[index].text):
                        dict1["dewy_decimal_class"] = dewy.findChild("dd").text.replace("\n","").replace(" ","")

                    if  ("Number of pages" in dewy.find_all("dt")[index].text):
                        dict1["num_of_pages"] = dewy.find_all("dd")[index].text.replace("\n","").replace(" ","").replace(";","")
                    if ("Pagination" in dewy.find_all("dt")[index].text):
                        dict1["pagination"] = dewy.find_all("dd")[index].text.replace(" ;\n\n","")

            except:
                pass
    
        except:
            pass
    

    
    return dict1

In [434]:
import json
for index, item in enumerate(bookURLs):

    delayTime = getRobotDelay(index,delayTime)
    
    print("\n\nzzzzz Sleeping for ",delayTime," sec(s), from Robot.txt")
    time.sleep(delayTime)
    print("ooooo Awake Now")
    
    page = requests.get(bookURLs[index]) #url
    soup = BeautifulSoup(page.content, "html.parser")
    
    csvHere = getBookDetails(soup)
    try:
        with open(bookURLs[index].split("https://openlibrary.org/works/")[1]+'.json', 'w') as outfile:
            json.dump(csvHere, outfile)
            
        print("Saved CSV file as\'",bookURLs[index].split("https://openlibrary.org/works/")[1]+'.json\'')
    except Exception as e:
        print("Couldn't export to CSV for Book (URL)"+bookURLs[index]+"\nStack Trace Here:\n")
        print(e)
        
    #break after 4 saves
    if (index == 3): break

just got robot.txt... current delay time:  0.5


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
Saved CSV file as' OL6729940W.json'


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
Saved CSV file as' OL6729938W.json'


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
Saved CSV file as' OL6729935W.json'


zzzzz Sleeping for  0.5  sec(s), from Robot.txt
ooooo Awake Now
Saved CSV file as' OL6729930W.json'
